In [28]:
import keras
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.layers import Dense

from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical

from keras.models import Sequential, Model 
from keras.utils import np_utils
from keras.layers import  BatchNormalization,Dense, Conv2D, Convolution2D, MaxPooling2D, Dropout, Flatten, TimeDistributed, InputLayer, LSTM
from keras.layers import Input, Reshape, Activation, add, Add
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from keras import optimizers

from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import f1_score 
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
file=open("mindbigdata.txt","r")
data=[]

In [3]:
for line in file.readlines():
    data.append(line.replace("\t",",").split(","))

In [4]:
eeg=pd.DataFrame(data)

In [5]:
eeg=eeg.dropna(axis=1)

In [11]:
X=eeg.drop(eeg.columns[[0,1,2,3,4,5]],axis='columns')
y=eeg[4]

In [34]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,0.592058,0.589073,0.587879,0.590147,0.592177,0.592774,0.592416,0.591222,0.591102,0.592177,...,0.586923,0.586326,0.587759,0.589073,0.58955,0.590028,0.589431,0.588595,0.58752,0.586446
1,0.613908,0.610684,0.610445,0.613311,0.616177,0.61558,0.613311,0.614027,0.616057,0.616415,...,0.614027,0.61343,0.615221,0.617012,0.619042,0.619878,0.617729,0.616296,0.617371,0.618206
2,0.62537,0.623102,0.622147,0.622505,0.623938,0.62537,0.624654,0.622982,0.623102,0.623818,...,0.619162,0.620356,0.622027,0.623102,0.623579,0.622505,0.621072,0.620833,0.620236,0.618684
3,0.548357,0.54788,0.546686,0.545253,0.546208,0.54979,0.551103,0.549074,0.547163,0.546327,...,0.548954,0.548238,0.549074,0.549909,0.550148,0.550506,0.550506,0.549671,0.547521,0.546447
4,0.615938,0.616057,0.615221,0.615938,0.617848,0.618326,0.616774,0.615818,0.616057,0.616893,...,0.618087,0.618326,0.618206,0.617968,0.619162,0.619878,0.619281,0.618565,0.617251,0.616296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910471,0.622982,0.622505,0.622266,0.620236,0.620833,0.62537,0.624773,0.619759,0.618206,0.619162,...,0.63146,0.633251,0.632296,0.628117,0.629908,0.632773,0.630982,0.629549,0.629908,0.630863
910472,0.55373,0.553133,0.555282,0.554685,0.551223,0.550865,0.553969,0.557073,0.556118,0.55182,...,0.572954,0.576416,0.576655,0.574745,0.574625,0.571879,0.570566,0.574506,0.573431,0.569372
910473,0.634206,0.634206,0.6354,0.635042,0.634684,0.634922,0.634803,0.635758,0.635878,0.632893,...,0.646146,0.648295,0.646743,0.644594,0.645788,0.644833,0.643519,0.645668,0.64543,0.643997
910474,0.488179,0.487702,0.487582,0.486985,0.487463,0.488538,0.488299,0.488776,0.489732,0.487463,...,0.515283,0.517313,0.517194,0.514806,0.513612,0.51194,0.512537,0.514567,0.509552,0.505851


In [14]:
X=X.values
y=y.values

In [17]:
scaler=MinMaxScaler()
X[:]=scaler.fit_transform(X[:])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None, shuffle=True, stratify=y)

In [21]:
y_train = to_categorical(y_train, 11)
y_test = to_categorical(y_test, 11)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(637333, 136) (273143, 136) (637333, 11) (273143, 11)


In [30]:
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [31]:
def model_create(eeg_data):
    eeg_input=Input(shape=(136,)) #입력 정의
    
    eeg_output = Dense(50, activation='relu')(eeg_input)
    eeg_output = Dropout(0.5)(eeg_output)
    eeg_output = Dense(150, activation='relu')(eeg_input)
    eeg_output = Dropout(0.5)(eeg_output)
    eeg_output = Dense(150, activation='relu')(eeg_input)
    eeg_output = Dropout(0.5)(eeg_output)
    eeg_output = Dense(200, activation='relu')(eeg_output)
    
    model = Dense(11, activation='sigmoid')(eeg_output)
    
    model = keras.models.Model(inputs=eeg_input, outputs=model) 
    
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy', precision, recall, f1score])
    
    return model 

model=model_create(X)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 136)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 150)               20550     
_________________________________________________________________
dropout_12 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 200)               30200     
_________________________________________________________________
dense_20 (Dense)             (None, 11)                2211      
Total params: 52,961
Trainable params: 52,961
Non-trainable params: 0
_________________________________________________________________


In [32]:
hist=model.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_test,y_test))

Train on 637333 samples, validate on 273143 samples
Epoch 1/50
637333/637333 [==============================] - 261s 409us/step - loss: 0.2972 - accuracy: 0.9090 - precision: 3.2964e-05 - recall: 4.2364e-05 - f1score: 2.3168e-05 - val_loss: 0.2969 - val_accuracy: 0.9093 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00
Epoch 2/50
637333/637333 [==============================] - 261s 409us/step - loss: 0.2967 - accuracy: 0.9090 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: 0.0000e+00 - val_loss: 0.2967 - val_accuracy: 0.9093 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00
Epoch 3/50
637333/637333 [==============================] - 260s 408us/step - loss: 0.2967 - accuracy: 0.9090 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1score: 0.0000e+00 - val_loss: 0.2968 - val_accuracy: 0.9093 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00
Epoch 4/50
637333/637333 [============================

KeyboardInterrupt: 

In [ ]:
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

#results = model.evaluate(X_test, y_test)
print('Test accuracy: ', results)

_loss, _acc, _precision, _recall, _f1score = model.evaluate(X_test, y_test)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _precision, _recall, _f1score))

In [ ]:
fig, rec_ax = plt.subplots()

pre_ax = rec_ax.twinx()

rec_ax.plot(hist.history['recall'], 'y', label='train recall')
rec_ax.plot(hist.history['val_recall'], 'r', label='val recall')

pre_ax.plot(hist.history['precision'], 'b', label='train precision')
pre_ax.plot(hist.history['val_precision'], 'g', label='val precision')

rec_ax.set_xlabel('epoch')
rec_ax.set_ylabel('recall')
pre_ax.set_ylabel('precision')

rec_ax.legend(loc='upper left')
pre_ax.legend(loc='lower left')

plt.show()